In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [2]:
def sigmoid(Z):
    A = (1 + np.exp(-Z))**(-1)
    return A

def sigmoid_derv(Z):
    A = sigmoid(Z)
    return A*(1-A)

In [3]:
def tanh(Z):
    return np.multiply((np.exp(Z)-np.exp(-Z)), (np.exp(Z)+np.exp(-Z))**-1)

def tanh_derv(Z):
    A = sigmoid(Z)
    return 1-(A**2)

In [4]:
def relu(Z):
    return np.maximum(0, Z)

def relu_derv(Z):
    return (Z>=0).astype(int)

In [5]:
def leaky_relu(Z, leak):
    return np.maximum(leak*Z, Z)

def leaky_relu_derv(Z, leak):
    x = (Z>=0).astype(int)
    x[x==0] = leak
    return x

In [6]:
class NeuralNetwork:
    def __init__(self, activation_list=[sigmoid], activation_derv_list=[sigmoid]):
        self.activation_list = [None] + activation_list
        self.activation_derv_list = [None] + activation_derv_list
        self.loss_history = []
        
    def random_initialize(self, num=0.01):
        for l in range(1, len(self.ndims)):
            self.W_list[l] = np.random.randn(self.ndims[l-1], self.ndims[l])*num
            self.dW_list[l] = np.random.randn(self.ndims[l-1], self.ndims[l])*num
            self.B_list[l] = np.random.randn(1, self.ndims[l])*num
            self.dB_list[l] = np.random.randn(1, self.ndims[l])*num
            
    def forward_prop(self):
        for l in range(1, len(self.ndims)):
            self.Z_list[l] = np.dot(self.A_list[l-1], self.W_list[l]) + self.B_list[l]
            self.A_list[l] = self.activation_list[l](self.Z_list[l])
            
    def backward_prop(self):
        for l in range(len(self.ndims)-1, 0, -1):
            if l==len(self.ndims)-1:
                self.dA_list[l] = (self.A_list[l]-self.Y)/(self.A_list[l]*(1-self.A_list[l]))
            else:
                self.dA_list[l] = np.dot(self.dZ_list[l+1], (self.W_list[l+1].T))
            self.dZ_list[l] = self.dA_list[l]*self.activation_derv_list[l](self.Z_list[l])
            self.dW_list[l] = np.dot((self.A_list[l-1].T), (self.dZ_list[l]))
            self.dB_list[l] = np.sum(self.dZ_list[l], axis=0, keepdims=True)
                
    def update(self):
        for l in range(1, len(self.ndims)):
            self.W_list[l] = self.W_list[l] - (self.alpha*self.dW_list[l])/self.m
            self.B_list[l] = self.B_list[l] - (self.alpha*self.dB_list[l])/self.m
            
    def loss(self, A, Y):
        return (-(Y*np.log(A) + (1-Y)*np.log(1-A))).sum()/self.m
        
    def fit(self, X, Y, iterations, alpha, ndims=[]):
        
        self.X = X
        self.Y = Y
        self.iterations = iterations
        self.alpha = alpha
        
        
        self.m = self.X.shape[0]
        self.n0 = self.X.shape[1]
        self.nlast = 1 if len(self.Y.shape)==1 else self.Y.shape[1]
        self.Y = self.Y.reshape((self.Y.shape[0], self.nlast))

        self.ndims = [self.n0] + ndims + [self.nlast]
        self.A_list = [self.X] + [None]*(len(self.ndims)-1)
        self.dA_list = [None]*len(self.ndims)
        self.Z_list = [None]*len(self.ndims)
        self.dZ_list = [None]*len(self.ndims)
        self.W_list = [None]*len(self.ndims)
        self.dW_list = [None]*len(self.ndims)
        self.B_list = [None]*len(self.ndims)
        self.dB_list = [None]*len(self.ndims)
        
        self.random_initialize()
        
        for _ in range(self.iterations):
            self.forward_prop()
            self.backward_prop()
            self.update()
            self.loss_history += self.loss(self.A_list[-1], self.Y)
    
    def predict(self, X, threshold=0.5):
        for l in range(1, len(self.ndims)):
            Z = np.dot(X, self.W_list[l]) + self.B_list[l]
            X = self.activation_list[l](Z)
        return X

In [7]:
data = load_breast_cancer()
X = data.data
Y = data.target
scaler=StandardScaler()
X=scaler.fit_transform(X)

In [8]:
model = NeuralNetwork([tanh, sigmoid], [tanh_derv, sigmoid_derv])

In [9]:
model.fit(X, Y, 100, 1, [10])

In [10]:
(pd.Series(model.predict(X).T[0]>=0.5) == pd.Series(Y)).sum()/len(Y)

0.9859402460456942